# Requerimientos

In [2]:
!pip install yt-dlp -q

In [3]:
!pip install SpeechRecognition -q

In [4]:
!pip install pydub -q

In [5]:
!pip install dask -q

In [6]:
!pip install openai-whisper -q

In [40]:
!pip install vosk -q

# Zona de Importaciones

In [41]:
import pandas as pd
import shutil
import tempfile
import yt_dlp
import speech_recognition as sr
import whisper
from pydub import AudioSegment
import dask
from dask import delayed
from dask.diagnostics import ProgressBar
import dask.dataframe as dd
import time
import concurrent.futures
import os

from vosk import Model, KaldiRecognizer
import wave
import json

# Variables de entorno

In [60]:
COOKIES_FILE_PATH = '../notebooks/conf/youtube.com_cookies.txt'
#COOKIES_FILE_PATH = 'C:/Users/WilmarAl/Documents/00-Personal/01 - Maestria/00 - UNIVERSIDAD CENTRAL/4 Semestre/Proyecto/uc-pry-mad-analisis-sentimiento/notebooks/conf/youtube.com_cookies.txt'
INPUT_FILE_PATH = '../notebooks/data/canciones limpias.xlsx'
INPUT_FILE_PATH_2 = '../notebooks/data/canciones_urls.xlsx'
OUTPUT_PATH = '../notebooks/data/'
AUDIO_PATH = 'C:/Users/WilmarAl/Documents/00-Personal/01 - Maestria/00 - UNIVERSIDAD CENTRAL/4 Semestre/Proyecto/uc-pry-mad-analisis-sentimiento/notebooks/data/audios/'

In [61]:
with open(COOKIES_FILE_PATH, 'r', encoding='utf-8') as archivo:
    contenido = archivo.read()

print(contenido)

# Netscape HTTP Cookie File
# http://curl.haxx.se/rfc/cookie_spec.html
# This is a generated file!  Do not edit.

.youtube.com	TRUE	/	TRUE	1776170590	__Secure-3PAPISID	01f8IHmR8niTVDo7/A5oGtYbF0FW7H1xDb
.youtube.com	TRUE	/	TRUE	1776170590	__Secure-3PSID	g.a000ughC731s0rblxbidyYUDv-35aDoEFSi5EIfmC7J5548fpYFEIwmWIHBGq8hxOX8lvd7sagACgYKAVASARQSFQHGX2MiHjg2xyuIA8M2D2rcVMO_URoVAUF8yKqwEhPBG2LsmnZVSar-rlhl0076
.youtube.com	TRUE	/	TRUE	1776170590	LOGIN_INFO	AFmmF2swRQIgQIT-2FSNCxMbku0Rm8cpD2tpVDfJb_HDCCpskt8PkYICIQClp78dcWfAr4VA_83JhT35MKuKIL0vDScduhtKo8NvYw:QUQ3MjNmd0JCbUZsR2ZKZzB3YVdYNnFQLVNpWHRPbkVHdnpVclpNa1RpRkZQcDRzMUVaWGd4R0RNTV82YmdocjIwWlowczY1RE8zc3pNaEdkRk8xZ0xRYm15Q0I0MXlyQWRnYk4xR0prcm5UeE9OdWN1LTNvMzhpRlhQaS1xMlNYTFNwa0QzYkpaM081c0xiYjhTZGZ0WnloX3VrWVpuT2ZR
.youtube.com	TRUE	/	FALSE	0	wide	1
.youtube.com	TRUE	/	TRUE	1776195035	PREF	f6=40000080&f4=4000000&tz=America.Bogota&f5=30000&f7=100
.youtube.com	TRUE	/	TRUE	1773170456	__Secure-1PSIDTS	sidts-CjEBEJ3XVw9tJEeF3UsNbOJdgktVLRRcr

# Cargar Dataframe

In [62]:
df = pd.read_excel(INPUT_FILE_PATH_2)

In [63]:
df = df[['TITULO','AÑO','COMPOSITOR','UBICACIÓN','DEPARTAMENTO','URL']]

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383711 entries, 0 to 383710
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   TITULO        383347 non-null  object
 1   AÑO           383711 non-null  int64 
 2   COMPOSITOR    383711 non-null  object
 3   UBICACIÓN     383711 non-null  object
 4   DEPARTAMENTO  383711 non-null  object
 5   URL           1202 non-null    object
dtypes: int64(1), object(5)
memory usage: 17.6+ MB


In [65]:
df = df.dropna().copy()

In [66]:
df['AUDIO_NAME'] = df['TITULO']+'_'+df['AÑO'].astype(str)+'_'+df['COMPOSITOR']

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1202 entries, 0 to 377914
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   TITULO        1202 non-null   object
 1   AÑO           1202 non-null   int64 
 2   COMPOSITOR    1202 non-null   object
 3   UBICACIÓN     1202 non-null   object
 4   DEPARTAMENTO  1202 non-null   object
 5   URL           1202 non-null   object
 6   AUDIO_NAME    1202 non-null   object
dtypes: int64(1), object(6)
memory usage: 75.1+ KB


# Particionar en dataframes por año

In [68]:
df_1990 = df.query('AÑO == 1990') # 3.028 Canciones
df_1991 = df.query('AÑO == 1991') # 2.139 Canciones
df_1992 = df.query('AÑO == 1992') # 2.524 Canciones
df_1993 = df.query('AÑO == 1993') # 2.295 Canciones
df_1994 = df.query('AÑO == 1994') # 2.542 Canciones
df_1995 = df.query('AÑO == 1995') # 3.516 Canciones
df_1996 = df.query('AÑO == 1996') # 4.031 Canciones
df_1997 = df.query('AÑO == 1997') # 3.977 Canciones
df_1998 = df.query('AÑO == 1998') # 5.467 Canciones
df_1999 = df.query('AÑO == 1999') # 5.608 Canciones
df_2000 = df.query('AÑO == 2000') # 8.268 Canciones
df_2001 = df.query('AÑO == 2001') # 5.657 Canciones
df_2002 = df.query('AÑO == 2002') # 6.834 Canciones
df_2003 = df.query('AÑO == 2003') # 7.714 Canciones
df_2004 = df.query('AÑO == 2004') # 8.795 Canciones
df_2005 = df.query('AÑO == 2005') # 10.218 Canciones
df_2006 = df.query('AÑO == 2006') # 11.601 Canciones
df_2007 = df.query('AÑO == 2007') # 12.085 Canciones
df_2008 = df.query('AÑO == 2008') # 14.446 Canciones
df_2009 = df.query('AÑO == 2009') # 13.847 Canciones
df_2010 = df.query('AÑO == 2010') # 18.299 Canciones
df_2011 = df.query('AÑO == 2011') # 14.784 Canciones
df_2012 = df.query('AÑO == 2012') # 17.175 Canciones
df_2013 = df.query('AÑO == 2013') # 15.052 Canciones
df_2014 = df.query('AÑO == 2014') # 17.497 Canciones
df_2015 = df.query('AÑO == 2015') # 17.836 Canciones
df_2016 = df.query('AÑO == 2016') # 18.376 Canciones
df_2017 = df.query('AÑO == 2017') # 19.377 Canciones
df_2018 = df.query('AÑO == 2018') # 20.167 Canciones
df_2019 = df.query('AÑO == 2019') # 20.994 Canciones
df_2020 = df.query('AÑO == 2020') # 23.691 Canciones
df_2021 = df.query('AÑO == 2021') # 18.345 Canciones
df_2022 = df.query('AÑO == 2022') # 16.239 Canciones
df_2023 = df.query('AÑO == 2023') # 11.278 Canciones

# Definición de las funciones

In [69]:
def limpiar_texto(text):
  '''
    Limpiar el texto
  '''
  text = text.replace('\n', ' ') # Saltos de línea
  text = text.replace('\t', ' ') # Tabuladores
  text = text.replace('á', 'a')
  text = text.replace('é', 'e')
  text = text.replace('í', 'i')
  text = text.replace('ó', 'o')
  text = text.replace('ú', 'u')
  text = text.lower() # Pasa el texto a minúsculas
  return text

In [44]:
def buscar_youtube_url(titulo, anio, compositor, ubicacion):
  query = f'Canción: {titulo} - Compositor: {compositor}'
  # Crear una copia temporal del archivo de cookies
  temp_cookies = tempfile.NamedTemporaryFile(delete=False)
  shutil.copy(COOKIES_FILE_PATH, temp_cookies.name)
  
  ydl_opts = {
    "quiet": True,
    'geo_bypass': True,
    'cookiefile': temp_cookies.name
    }
  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    try:
      result = ydl.extract_info(f"ytsearch:{query}", download=False)
      for entry in result["entries"]:
        if (entry.get("duration") and entry["duration"] > 60) and (entry.get("duration") and entry["duration"] < 480):
          return entry["webpage_url"]
      return result["webpage_url"]
    except:
      return None

In [45]:
def descargar_audio(url, audio_name):
  # Crear una copia temporal del archivo de cookies
  #print(f'EMPEZANDO DESCARGA: {url}')
  temp_cookies = tempfile.NamedTemporaryFile(delete=False)
  shutil.copy(COOKIES_FILE_PATH, temp_cookies.name)
  #print(f"Archivo de cookies copiado a: {temp_cookies.name}")

  ydl_opts = {
    "quiet": True,
    'format': 'bestaudio/best',
    'outtmpl': AUDIO_PATH+audio_name,
    'cookiefile': temp_cookies.name,
    'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    #'cookiesfrombrowser': ('chrome',),  # Usar cookies de Chrome
    'postprocessors': [{
      'key': 'FFmpegExtractAudio',
      'preferredcodec': 'wav',
      'preferredquality': '192',
    }],
    "prefer_ffmpeg": True
  }
  try:
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      ydl.download([url])
  except Exception as e:
    print(f'Ocurrió un error: {e}')
    
  # finally:
  #   temp_cookies.close()
  #   os.unlink(temp_cookies.name)  # Eliminar el archivo temporal después de su uso

In [47]:
def speech_recognition(audio_name):
  '''
    Extraer el texto del audio

    Parámetros:
    audio_name: Nombre del archivo de audio

    Retorna:
    text: Texo reconocido del audio en español
  '''
  try:
    audio_path = AUDIO_PATH+audio_name + '.wav' # Se construye la ruta completa para acceder al audio: La ruta ya está validada y si hay un audio allí con el nombre que ejecuto
    print(audio_path) # outhput: C:/Users/WilmarAl/Documents/00-Personal/01 - Maestria/00 - UNIVERSIDAD CENTRAL/4 Semestre/Proyecto/uc-pry-mad-analisis-sentimiento/notebooks/data/audios/A MI MANERA TE QUIERO_1990_IVAN ANTONIO OVALLE POVEDA.wav
    reconocedor = sr.Recognizer() # Libreria speech recognition
    audio = AudioSegment.from_wav(audio_path) 
    text = ''
    for i in range(0, len(audio), 30000):
      fragmento = audio[i:i+30000]
      temp_audio_path = AUDIO_PATH + audio_name + '_temp.wav'

      fragmento.export(temp_audio_path, format='wav')

      with sr.AudioFile(temp_audio_path) as fuente:
        audio_data = reconocedor.record(fuente)
        text_chunk = reconocedor.recognize_google(audio_data, language='es-ES')
        text += text_chunk + ' '
      
      os.remove(temp_audio_path)  # Eliminar archivo temporal
    
    os.remove(audio_path)

    if not text.strip():
      return None
    else:
      text = limpiar_texto(text)
      return text
  except Exception as e:
    print(f"Error al procesar el audio {audio_name}: {e}")
    return None

In [ ]:
def vosk_model(audio_name):
    audio = AUDIO_PATH+audio_name+'.wav' #'C:/Users/WilmarAl/Documents/00-Personal/01 - Maestria/00 - UNIVERSIDAD CENTRAL/4 Semestre/Proyecto/uc-pry-mad-analisis-sentimiento/notebooks/data/audios/A MI MANERA TE QUIERO_1990_IVAN ANTONIO OVALLE POVEDA.wav'
    model = Model("model-es")  # Descarga un modelo de Vosk
    wf = wave.open(audio, 'rb')
    rec = KaldiRecognizer(model, wf.getframerate())

    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            result = rec.Result()
            print(json.loads(result)["text"])
        else:
            print(rec.PartialResult())

    return rec.FinalResult()

In [73]:
def whisper_openai(audio_name):
    try:
        audio = AUDIO_PATH+audio_name+'.wav' #'C:/Users/WilmarAl/Documents/00-Personal/01 - Maestria/00 - UNIVERSIDAD CENTRAL/4 Semestre/Proyecto/uc-pry-mad-analisis-sentimiento/notebooks/data/audios/A MI MANERA TE QUIERO_1990_IVAN ANTONIO OVALLE POVEDA.wav'
        model = whisper.load_model("small")  # O usa "small", "medium", "large"
        result = model.transcribe(audio,  language="es")
        text = result["text"]
        if not text.strip():
            return None
            # text = vosk_model(audio_name)
            # if not text.strip():
            #     return None
            # else:
            #     text = limpiar_texto(text)
            #     return text
        else:
            os.remove(audio)
            text = limpiar_texto(text)
            return text
    except Exception as e:
        print(f"Error al procesar el audio {audio_name}: {e}")
        return None

In [27]:
text = whisper_openai('A MI MANERA TE QUIERO_1990_IVAN ANTONIO OVALLE POVEDA')

Error al procesar el audio A MI MANERA TE QUIERO_1990_IVAN ANTONIO OVALLE POVEDA: Failed to load audio: ffmpeg version 2025-02-26-git-99e2af4e78-essentials_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-dxva2 --enable-d3d11va --enable-d3d12va --enable-ffnvcodec --enable-libvpl --ena

c:\Users\WilmarAl\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [22]:
print(text)

 Oye es la sangre, suye mi vida y ser mi miedo de amor Yo sé que tú eres muy feliz, amor con las costumbres y tadina Y yo te amo sin saber por qué, siento las mías tan cuerdadida Pero a crece saber cómo en vez es mi valla y mi vida Y se va perdiendo la tercera y mesa de grandes amigos Siento en mi alma un desprendimiento y a mí me la estima Cuando quedo solo rodando en el valle, ¿cuál baja ruido? No le pide la gente, no le pide, no le pide Gracias a Dios, yo nací, seca el bata puri, que es la pesa de mi peso Y en mi destino, sin fin, quiero hacerse feliz, que a mi manera te quiero Y en mi destino, sin fin, quiero hacerte feliz y a mi manera te quiero Y a mi manera te quiero, y a mi manera voy a hacerte enamorar de mi frecío tremion ya mi manera te quiero ya mi manera voy a hacerte enamorar Emig Wales, voy hacerte enamorar Ya manpla te muese en la revada voy a hacerte enamorar, de un acordeón voy hacerte enamorar, de mi guitar voy a hacerte enamorar de mi canción y a mi manera te quiero

In [28]:
def get_url(titulo, anio, compositor, ubicacion):
  #time.sleep(2)
  url = buscar_youtube_url(titulo, anio, compositor, ubicacion)
  return url

In [51]:
def get_lyrics(url, audio_name):
  time.sleep(2)
  # print(f'DESCARGANDO AUDIO DE {audio_name}')
  descargar_audio(url, audio_name)
  # print(f'OPTENIENDO LETRA DE {audio_name}')
  #text = speech_recognition(audio_name)
  text = whisper_openai(audio_name)
  return text

In [42]:
letra_test = get_lyrics('https://www.youtube.com/@AlbeiroRincon/videos','Sangre en Mis Manos')

KeyboardInterrupt: 

In [27]:
print(letra_test)

None


# Paralelismo

## Buscar URL

In [19]:
# años = list(range(1990, 2024))

# # Recorrer el dataframe df por cada año para aplicar una función
# for año in años:
#     df_filtrado = df.query('AÑO == @año').copy()
#     if df_filtrado.empty:
#         continue

#     tasks_1 = [delayed(get_url)(titulo, anio, compositor, ubicacion) 
#                for titulo, anio, compositor, ubicacion in df_filtrado[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values
#                ]
    
#     with ProgressBar():
#         df_filtrado['YT_URL'] = dask.compute(*tasks_1, scheduler='threads')
    
#     # Actualizar el dataframe principal
#     df.update(df_filtrado)


In [20]:
# tasks_1990 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_1990[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_1991 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_1991[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_1992 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_1992[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_1993 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_1993[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_1994 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_1994[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_1995 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_1995[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_1996 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_1996[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_1997 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_1997[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_1998 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_1998[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_1999 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_1999[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2000 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2000[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2001 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2001[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2002 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2002[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2003 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2003[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2004 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2004[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2005 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2005[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2006 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2006[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2007 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2007[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2008 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2008[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2009 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2009[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2010 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2010[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2011 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2011[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2012 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2012[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2013 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2013[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2014 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2014[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2015 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2015[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2016 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2016[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2017 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2017[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2018 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2018[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2019 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2019[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2020 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2020[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2021 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2021[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2022 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2022[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]
# tasks_2023 = [delayed(get_url)(titulo, anio, compositor, ubicacion) for titulo, anio, compositor, ubicacion in df_2023[['TITULO', 'AÑO', 'COMPOSITOR', 'UBICACIÓN']].values]

In [ ]:
# print("Extrayendo URL's para 1990")
# with ProgressBar():
#   df_1990['YT_URL'] = dask.compute(*tasks_1990, scheduler='threads')
# print("Extrayendo URL's para 1991")
# with ProgressBar():
#   df_1991['YT_URL'] = dask.compute(*tasks_1991, scheduler='threads')
# print("Extrayendo URL's para 1992")
# with ProgressBar():
#   df_1992['YT_URL'] = dask.compute(*tasks_1992, scheduler='threads')
# print("Extrayendo URL's para 1993")
# with ProgressBar():
#   df_1993['YT_URL'] = dask.compute(*tasks_1993, scheduler='threads')
# print("Extrayendo URL's para 1994")
# with ProgressBar():
#   df_1994['YT_URL'] = dask.compute(*tasks_1994, scheduler='threads')
# print("Extrayendo URL's para 1995")
# with ProgressBar():
#   df_1995['YT_URL'] = dask.compute(*tasks_1995, scheduler='threads')
# print("Extrayendo URL's para 1996")
# with ProgressBar():
#   df_1996['YT_URL'] = dask.compute(*tasks_1996, scheduler='threads')
# print("Extrayendo URL's para 1997")
# with ProgressBar():
#   df_1997['YT_URL'] = dask.compute(*tasks_1997, scheduler='threads')
# print("Extrayendo URL's para 1998")
# with ProgressBar():
#   df_1998['YT_URL'] = dask.compute(*tasks_1998, scheduler='threads')
# print("Extrayendo URL's para 1999")
# with ProgressBar():
#   df_1999['YT_URL'] = dask.compute(*tasks_1999, scheduler='threads')
# print("Extrayendo URL's para 2000")
# with ProgressBar():
#   df_2000['YT_URL'] = dask.compute(*tasks_2000, scheduler='threads')
# print("Extrayendo URL's para 2001")
# with ProgressBar():
#   df_2001['YT_URL'] = dask.compute(*tasks_2001, scheduler='threads')
# print("Extrayendo URL's para 2002")
# with ProgressBar():
#   df_2002['YT_URL'] = dask.compute(*tasks_2002, scheduler='threads')
# print("Extrayendo URL's para 2003")
# with ProgressBar():
#   df_2003['YT_URL'] = dask.compute(*tasks_2003, scheduler='threads')
# print("Extrayendo URL's para 2004")
# with ProgressBar():
#   df_2004['YT_URL'] = dask.compute(*tasks_2004, scheduler='threads')
# print("Extrayendo URL's para 2005")
# with ProgressBar():
#   df_2005['YT_URL'] = dask.compute(*tasks_2005, scheduler='threads')
# print("Extrayendo URL's para 2006")
# with ProgressBar():
#   df_2006['YT_URL'] = dask.compute(*tasks_2006, scheduler='threads')
# print("Extrayendo URL's para 2007")
# with ProgressBar():
#   df_2007['YT_URL'] = dask.compute(*tasks_2007, scheduler='threads')
# print("Extrayendo URL's para 2008")
# with ProgressBar():
#   df_2008['YT_URL'] = dask.compute(*tasks_2008, scheduler='threads')
# print("Extrayendo URL's para 2009")
# with ProgressBar():
#   df_2009['YT_URL'] = dask.compute(*tasks_2009, scheduler='threads')
# print("Extrayendo URL's para 2010")
# with ProgressBar():
#   df_2010['YT_URL'] = dask.compute(*tasks_2010, scheduler='threads')
# print("Extrayendo URL's para 2011")
# with ProgressBar():
#   df_2011['YT_URL'] = dask.compute(*tasks_2011, scheduler='threads')
# print("Extrayendo URL's para 2012")
# with ProgressBar():
#   df_2012['YT_URL'] = dask.compute(*tasks_2012, scheduler='threads')
# print("Extrayendo URL's para 2013")
# with ProgressBar():
#   df_2013['YT_URL'] = dask.compute(*tasks_2013, scheduler='threads')
# print("Extrayendo URL's para 2014")
# with ProgressBar():
#   df_2014['YT_URL'] = dask.compute(*tasks_2014, scheduler='threads')
# print("Extrayendo URL's para 2015")
# with ProgressBar():
#   df_2015['YT_URL'] = dask.compute(*tasks_2015, scheduler='threads')
# print("Extrayendo URL's para 2016")
# with ProgressBar():
#   df_2016['YT_URL'] = dask.compute(*tasks_2016, scheduler='threads')
# print("Extrayendo URL's para 2017")
# with ProgressBar():
#   df_2017['YT_URL'] = dask.compute(*tasks_2017, scheduler='threads')
# print("Extrayendo URL's para 2018")
# with ProgressBar():
#   df_2018['YT_URL'] = dask.compute(*tasks_2018, scheduler='threads')
# print("Extrayendo URL's para 2019")
# with ProgressBar():
#   df_2019['YT_URL'] = dask.compute(*tasks_2019, scheduler='threads')
# print("Extrayendo URL's para 2020")
# with ProgressBar():
#   df_2020['YT_URL'] = dask.compute(*tasks_2020, scheduler='threads')
# print("Extrayendo URL's para 2021")
# with ProgressBar():
#   df_2021['YT_URL'] = dask.compute(*tasks_2021, scheduler='threads')
# print("Extrayendo URL's para 2022")
# with ProgressBar():
#   df_2022['YT_URL'] = dask.compute(*tasks_2022, scheduler='threads')
# print("Extrayendo URL's para 2023")
# with ProgressBar():
#   df_2023['YT_URL'] = dask.compute(*tasks_2023, scheduler='threads')

In [55]:
df_1990.info()

<class 'pandas.core.frame.DataFrame'>
Index: 141 entries, 7025 to 377908
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   TITULO        141 non-null    object
 1   AÑO           141 non-null    int64 
 2   COMPOSITOR    141 non-null    object
 3   UBICACIÓN     141 non-null    object
 4   DEPARTAMENTO  141 non-null    object
 5   URL           141 non-null    object
dtypes: int64(1), object(5)
memory usage: 7.7+ KB


In [56]:
df_1990.sample(2)

,TITULO,AÑO,COMPOSITOR,UBICACIÓN,DEPARTAMENTO,URL
158872,PERDIDO EN LAS COPAS,1990,ISRAEL MOTATTO,CAICEDONIA,VALLE DEL CAUCA,https://www.youtube.com/watch?v=HuhGtxegpH8
157672,A CHUPAR (A BESAR BESAR),1990,ISAAC VILLANUEVA MENDOZA,BARRANQUILLA,ATLANTICO,https://www.youtube.com/watch?v=iS2pl7Q-L6Y


In [57]:
df_filtered = df_1990[df_1990["YT_URL"].str.contains("https", na=False)]

KeyError: 'YT_URL'

In [ ]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 324 entries, 9 to 3024
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   TITULO        324 non-null    object
 1   AÑO           324 non-null    int64 
 2   COMPOSITOR    324 non-null    object
 3   UBICACIÓN     324 non-null    object
 4   DEPARTAMENTO  324 non-null    object
 5   YT_URL        324 non-null    object
dtypes: int64(1), object(5)
memory usage: 17.7+ KB


In [ ]:
df_filtered.sample(5)

,TITULO,AÑO,COMPOSITOR,UBICACIÓN,DEPARTAMENTO,YT_URL
2531,CON SABOR A SOL,1990,DANIEL DAVID HERRERA HERRERA,BOGOTA D.C.,BOGOTA D.C.,https://www.youtube.com/watch?v=uuf6iqmAyE0
960,CIELO AZUL,1990,DARIO CARCAMO GERRERO,BOGOTA D.C.,BOGOTA D.C.,https://www.youtube.com/watch?v=cuAw9dbeBXY
661,VUELVE A MI,1990,DANIEL JOSE ABADIA CARRASCO,BOGOTA D.C.,BOGOTA D.C.,https://www.youtube.com/watch?v=P6qYdAhKCSs
321,MILAGRO DE DIOS,1990,ROMUALDO LUIS BRITO LOPEZ,BOGOTA D.C.,BOGOTA D.C.,https://www.youtube.com/watch?v=hicGK5AKhCU
2874,COPLAS HERIDAS,1990,HERNÁN ALFONSO URBINA JOIRO,CARTAGENA,BOLIVAR,https://www.youtube.com/watch?v=ZvuYFY8EO2E


In [ ]:
# df_sampled.to_excel(f"{OUTPUT_PATH}canciones con URL.xlsx", index=False)

## Buscar Audio y Extraer la Letra

In [ ]:
# tasks_1990 = [delayed(get_lyrics)(url) for url in df_1990['URL']]
# tasks_1991 = [delayed(get_lyrics)(url) for url in df_1991['URL']]
# tasks_1992 = [delayed(get_lyrics)(url) for url in df_1992['URL']]
# tasks_1993 = [delayed(get_lyrics)(url) for url in df_1993['URL']]
# tasks_1994 = [delayed(get_lyrics)(url) for url in df_1994['URL']]
# tasks_1995 = [delayed(get_lyrics)(url) for url in df_1995['URL']]
# tasks_1996 = [delayed(get_lyrics)(url) for url in df_1996['URL']]
# tasks_1997 = [delayed(get_lyrics)(url) for url in df_1997['URL']]
# tasks_1998 = [delayed(get_lyrics)(url) for url in df_1998['URL']]
# tasks_1999 = [delayed(get_lyrics)(url) for url in df_1999['URL']]
# tasks_2000 = [delayed(get_lyrics)(url) for url IN df_2000['URL']]
# tasks_2001 = [delayed(get_lyrics)(url) for url in df_2001['URL']]
# tasks_2002 = [delayed(get_lyrics)(url) for url in df_2002['URL']]
# tasks_2003 = [delayed(get_lyrics)(url) for url in df_2003['URL']]
# tasks_2004 = [delayed(get_lyrics)(url) for url in df_2004['URL']]
# tasks_2005 = [delayed(get_lyrics)(url) for url in df_2005['URL']]
# tasks_2006 = [delayed(get_lyrics)(url) for url in df_2006['URL']]
# tasks_2007 = [delayed(get_lyrics)(url) for url in df_2007['URL']]
# tasks_2008 = [delayed(get_lyrics)(url) for url in df_2008['URL']]
# tasks_2009 = [delayed(get_lyrics)(url) for url in df_2009['URL']]
# tasks_2010 = [delayed(get_lyrics)(url) for url in df_2010['URL']]
# tasks_2011 = [delayed(get_lyrics)(url) for url in df_2011['URL']]
# tasks_2012 = [delayed(get_lyrics)(url) for url in df_2012['URL']]
# tasks_2013 = [delayed(get_lyrics)(url) for url in df_2013['URL']]
# tasks_2014 = [delayed(get_lyrics)(url) for url in df_2014['URL']]
# tasks_2015 = [delayed(get_lyrics)(url) for url in df_2015['URL']]
# tasks_2016 = [delayed(get_lyrics)(url) for url in df_2016['URL']]
# tasks_2017 = [delayed(get_lyrics)(url) for url in df_2017['URL']]
# tasks_2018 = [delayed(get_lyrics)(url) for url in df_2018['URL']]
# tasks_2019 = [delayed(get_lyrics)(url) for url in df_2019['URL']]
# tasks_2020 = [delayed(get_lyrics)(url) for url in df_2020['URL']]
# tasks_2021 = [delayed(get_lyrics)(url) for url in df_2021['URL']]
# tasks_2022 = [delayed(get_lyrics)(url) for url in df_2022['URL']]
# tasks_2023 = [delayed(get_lyrics)(url) for url in df_2023['URL']]

In [ ]:
# df_1990.info()

In [ ]:
# with ProgressBar(): 
    # df_1990['LETRA'] = df_1990['URL'].apply(lambda x: get_lyrics(x))

In [74]:
tasks_2 = [delayed(get_lyrics)(url, audio_name) for url, audio_name in df[['URL','AUDIO_NAME']].values]
#tasks_2 = [delayed(get_lyrics)(url, audio_name) for url, audio_name in df_1990[['URL','AUDIO_NAME']].values]
# df['LETRA'] = dask.compute(*tasks_2, scheduler='threads')

In [75]:
with ProgressBar(): 
    df['LETRA'] = dask.compute(*tasks_2, scheduler='threads')

[########################################] | 100% Completed | 55m 8ss


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1202 entries, 0 to 377914
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   TITULO        1202 non-null   object
 1   AÑO           1202 non-null   int64 
 2   COMPOSITOR    1202 non-null   object
 3   UBICACIÓN     1202 non-null   object
 4   DEPARTAMENTO  1202 non-null   object
 5   URL           1202 non-null   object
 6   AUDIO_NAME    1202 non-null   object
 7   LETRA         23 non-null     object
dtypes: int64(1), object(7)
memory usage: 84.5+ KB


In [78]:
print(df['LETRA'].sample(20))

289383                                                 None
344815                                                 None
15441                                                  None
311832     como tengo que buscarte a tu aparecimiento? v...
159456                                                 None
159465                                                 None
159477                                                 None
159393                                                 None
80589                                                  None
265834     ¡cuidado! ¡cuidado! ¡pone el pizzi! como yo t...
15427      estas son mis novias, mis novias que pueden s...
260812                                                 None
80595                                                  None
15391                                                  None
344687                                                 None
344672                                                 None
38021                                   

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   TITULO        0 non-null      object
 1   AÑO           0 non-null      int64 
 2   COMPOSITOR    0 non-null      object
 3   UBICACIÓN     0 non-null      object
 4   DEPARTAMENTO  0 non-null      object
 5   URL           0 non-null      object
 6   AUDIO_NAME    0 non-null      object
 7   LETRA         0 non-null      object
dtypes: int64(1), object(7)
memory usage: 0.0+ bytes


In [ ]:
# df.to_excel(f"{OUTPUT_PATH}canciones_letra.xlsx", index=False)